# Baseline Model using Opcode Entropy for binary

## Set up

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import json
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score


In [3]:
PATH = Path.cwd().parents[2]
DATA_PATH = os.path.join(PATH, 'data/processed/opcode_entropy')

## List Traditional ML model to compare

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

models = {
  "LogisticRegression()": LogisticRegression(),
  "DecisionTreeClassifier()": DecisionTreeClassifier(),
  "RandomForestClassifier()": RandomForestClassifier(),
  "AdaBoostClassifier()": AdaBoostClassifier(),
  "ExtraTreesClassifier()": ExtraTreesClassifier(),
  "XGBClassifier()": XGBClassifier(),
  "LGBMClassifier()": LGBMClassifier(),
  "SVC()": SVC(),
  "GaussianNB()": GaussianNB(),
  "KNeighborsClassifier()": KNeighborsClassifier(),
  "SGDClassifier()": SGDClassifier(),
  "MLPClassifier()": MLPClassifier(),
}

## Load Dataset

In [5]:
df = pd.read_csv(os.path.join(DATA_PATH, 'dataset.csv')).set_index('address')

with open(os.path.join(DATA_PATH, 'features.json'), "r") as f:
    features = json.load(f)

with open(os.path.join(DATA_PATH, 'labels.json'), "r") as f:
    labels = json.load(f)

In [6]:
df.head()

,mint,leak,limit,opcode_entropy
address,,,,
0x93023f1d3525e273f291b6f76d2f5027a39bf302,1,0,1,3.588720
0x2753dce37a7edb052a77832039bcc9aa49ad8b25,0,0,1,3.605842
0x94b7d24552933f50a5a5705c446528806dcea381,0,0,0,3.941392
0xe0b9d4146aad6936cbfcbe4dae47e34aab96b093,0,0,0,3.566705
0x10f6f2b97f3ab29583d9d38babf2994df7220c21,1,0,1,3.577189


In [7]:
features[:5]

['opcode_entropy']

In [8]:
labels[:5]

['mint', 'leak', 'limit']

## Prepare Features and Labels

In [9]:
X = df[features]
y_mint = df[['mint']]
y_leak = df[['leak']]
y_limit = df[['limit']]

## Train/Test Split

In [10]:
X_mint_train, X_mint_test, y_mint_train, y_mint_test = train_test_split(
    X, y_mint, test_size=0.2, random_state=42
)

X_leak_train, X_leak_test, y_leak_train, y_leak_test = train_test_split(
    X, y_leak, test_size=0.2, random_state=42
)

X_limit_train, X_limit_test, y_limit_train, y_limit_test = train_test_split(
    X, y_limit, test_size=0.2, random_state=42
)

## Run all models and collect reports

In [11]:
def get_report_all_ml(X_train, y_train, X_test, y_test):
    report_list = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Generate classification report (as dict)
        report_dict = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

        # Average scores across all labels (macro average)
        avg_scores = report_dict["macro avg"]

        report_list.append({
            "Model": name,
            "Precision": avg_scores["precision"],
            "Recall": avg_scores["recall"],
            "F1-score": avg_scores["f1-score"]
        })

    df_report = pd.DataFrame(report_list)
    df_report = df_report.sort_values("F1-score", ascending=False).reset_index(drop=True)

    return df_report

In [12]:
def display_cm(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1])
    return cm_display

## Show report

In [13]:
data = {
  'mint': {
    "X_train": X_mint_train,
    "X_test": X_mint_test,
    "y_train": y_mint_train,
    "y_test": y_mint_test
  },
  'leak': {
    "X_train": X_leak_train,
    "X_test": X_leak_test,
    "y_train": y_leak_train,
    "y_test": y_leak_test
  },
  'limit': {
    "X_train": X_limit_train,
    "X_test": X_limit_test,
    "y_train": y_limit_train,
    "y_test": y_limit_test
  }
}

reports = []

for d in data.values():
    X_train = d['X_train']
    X_test = d['X_test']
    y_train = d['y_train']
    y_test = d['y_test']
    df_report = get_report_all_ml(X_train, y_train, X_test, y_test)
    reports.append(df_report)

[LightGBM] [Info] Number of positive: 14, number of negative: 41
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000115 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20
[LightGBM] [Info] Number of data points in the train set: 55, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.254545 -> initscore=-1.074515
[LightGBM] [Info] Start training from score -1.074515
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

In [14]:
reports[0]

,Model,Precision,Recall,F1-score
0,XGBClassifier(),0.633333,0.625000,0.625668
1,LogisticRegression(),0.285714,0.500000,0.363636
2,LGBMClassifier(),0.285714,0.500000,0.363636
3,SVC(),0.285714,0.500000,0.363636
4,MLPClassifier(),0.285714,0.500000,0.363636
5,SGDClassifier(),0.285714,0.500000,0.363636
6,GaussianNB(),0.285714,0.500000,0.363636
7,DecisionTreeClassifier(),0.357143,0.354167,0.353846
8,AdaBoostClassifier(),0.357143,0.354167,0.353846
9,RandomForestClassifier(),0.357143,0.354167,0.353846


In [15]:
reports[1]

,Model,Precision,Recall,F1-score
0,LogisticRegression(),0.392857,0.5,0.44
1,DecisionTreeClassifier(),0.392857,0.5,0.44
2,RandomForestClassifier(),0.392857,0.5,0.44
3,AdaBoostClassifier(),0.392857,0.5,0.44
4,ExtraTreesClassifier(),0.392857,0.5,0.44
5,XGBClassifier(),0.392857,0.5,0.44
6,LGBMClassifier(),0.392857,0.5,0.44
7,SVC(),0.392857,0.5,0.44
8,GaussianNB(),0.392857,0.5,0.44
9,KNeighborsClassifier(),0.392857,0.5,0.44


In [16]:
reports[2]

,Model,Precision,Recall,F1-score
0,XGBClassifier(),0.727273,0.666667,0.562500
1,SVC(),0.575000,0.566667,0.497436
2,KNeighborsClassifier(),0.575000,0.566667,0.497436
3,LGBMClassifier(),0.428571,0.422222,0.416667
4,SGDClassifier(),0.321429,0.500000,0.391304
5,ExtraTreesClassifier(),0.400000,0.411111,0.353846
6,RandomForestClassifier(),0.400000,0.411111,0.353846
7,DecisionTreeClassifier(),0.400000,0.411111,0.353846
8,AdaBoostClassifier(),0.416667,0.455556,0.326203
9,LogisticRegression(),0.178571,0.500000,0.263158


## MLP

### Build model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError

2025-07-15 15:08:35.278722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
def Model(input_dim, output_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(512, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.4),

        Dense(256, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.3),

        Dense(128, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.2),

        Dense(output_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-6),
        metrics=['accuracy']
    )
    return model

In [19]:
model = Model(input_dim=X.shape[1], output_dim=1)

### Train

In [20]:
model.fit(X_mint_train, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 299ms/step - accuracy: 0.4129 - loss: 0.7900 - val_accuracy: 0.1818 - val_loss: 0.7198 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3617 - loss: 0.9070 - val_accuracy: 0.1818 - val_loss: 0.7186 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4081 - loss: 0.9443 - val_accuracy: 0.1818 - val_loss: 0.7178 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2955 - loss: 0.9665 - val_accuracy: 0.1818 - val_loss: 0.7176 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5104 - loss: 0.8141 - val_accuracy: 0.1818 - val_loss: 0.7161 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.4848 - loss: 0.7913 - val_accuracy: 0.1818 - val_loss: 0.7149 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2907 - loss: 0.8053 - 

### Predict

In [21]:
y_mint_test_prob = model.predict(X_mint_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step


In [22]:
def tune_thresholds(y_true, y_pred_prob, metric='f1'):
    y_true = np.asarray(y_true)          # Fix: convert to NumPy
    y_pred_prob = np.asarray(y_pred_prob)

    best_thresholds = []
    best_scores = []

    for i in range(y_true.shape[1]):
        label_true = y_true[:, i]
        label_probs = y_pred_prob[:, i]  # Fix here too

        thresholds = np.linspace(0.0, 1.0, 101)
        scores = []

        for t in thresholds:
            label_pred = (label_probs >= t).astype(int)
            if metric == 'f1':
                score = f1_score(label_true, label_pred, zero_division=0)
            scores.append(score)

        best_t = thresholds[np.argmax(scores)]
        best_score = np.max(scores)

        best_thresholds.append(best_t)
        best_scores.append(best_score)

        print(f"Label {i}: Best threshold = {best_t:.2f}, Best {metric} = {best_score:.4f}")

    return best_thresholds, best_scores


In [23]:
best_thresholds, _ = tune_thresholds(y_mint_test, y_mint_test_prob)

Label 0: Best threshold = 0.00, Best f1 = 0.6000


In [24]:
y_mint_pred = (model.predict(X_mint_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


### Report

In [25]:
print(classification_report(y_mint_test, y_mint_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.43      1.00      0.60         6

    accuracy                           0.43        14
   macro avg       0.21      0.50      0.30        14
weighted avg       0.18      0.43      0.26        14



## Autoencoder + MPL

In [26]:
def Autoencoder(input_dim=256):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128),
        LeakyReLU(0.01),

        Dense(32),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(128),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(input_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(optimizer=Adam(1e-6), loss=MeanSquaredError())
    return model


In [27]:
autoencoder = Autoencoder(input_dim=X.shape[1])
autoencoder.fit(X_mint_train, X_mint_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 207ms/step - loss: 11.1203 - val_loss: 11.0363
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 11.0972 - val_loss: 11.0306
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 11.1415 - val_loss: 11.0253
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 11.0872 - val_loss: 11.0198
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 11.0163 - val_loss: 11.0140
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 11.1368 - val_loss: 11.0079
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 11.0628 - val_loss: 11.0021
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - loss: 11.0883 - val_loss: 10.9964
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 11.1218 - val_loss: 10.9902
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 11.1069 - val_loss: 10.9852
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 11.1466 - val_loss: 10.9803
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 10.

In [28]:
X_train_encoded = autoencoder.predict(X_mint_train)
X_test_encoded = autoencoder.predict(X_mint_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [29]:
model = Model(input_dim=X_train_encoded.shape[1], output_dim=1)

In [30]:
model.fit(X_train_encoded, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 301ms/step - accuracy: 0.5559 - loss: 0.7555 - val_accuracy: 0.8182 - val_loss: 0.6995 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4792 - loss: 0.7343 - val_accuracy: 0.8182 - val_loss: 0.6988 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4697 - loss: 0.7378 - val_accuracy: 0.8182 - val_loss: 0.6980 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4848 - loss: 0.7706 - val_accuracy: 0.8182 - val_loss: 0.6971 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4848 - loss: 0.7729 - val_accuracy: 0.8182 - val_loss: 0.6963 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5966 - loss: 0.7017 - val_accuracy: 0.8182 - val_loss: 0.6957 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3409 - loss: 0.8246 - v

In [31]:
y_pred_prob = model.predict(X_test_encoded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step


In [32]:
best_thresholds, _ = tune_thresholds(y_mint_test, y_pred_prob)

Label 0: Best threshold = 0.00, Best f1 = 0.6000


In [33]:
y_pred = (model.predict(X_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


### Report

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.64      1.00      0.78         9

    accuracy                           0.64        14
   macro avg       0.32      0.50      0.39        14
weighted avg       0.41      0.64      0.50        14

